# Phase 2–3: Train DeepPTX

Load Parquet dataset, build tokenizers, create DataLoaders with curriculum sampling,
and train the Pointer-Generator Transformer.

**Works on:**
- Google Colab (T4/A100) — fastest, uses CUDA AMP
- Mac M1/M2/M3 — uses MPS acceleration (no AMP, still fast)
- CPU — slowest fallback

In [ ]:
# --- Run this cell first on Google Colab to clone the repo ---
import os
if os.path.exists("/content") and not os.path.exists("/content/DeepPTX"):
    !git clone https://github.com/ns-1456/DeepPTX.git /content/DeepPTX
    %cd /content/DeepPTX
    !pip install -q torch pyarrow pandas wandb tqdm

In [ ]:
import sys, os

# Detect environment
IN_COLAB = os.path.exists("/content")

if IN_COLAB:
    REPO_ROOT = "/content/DeepPTX"
else:
    REPO_ROOT = os.path.abspath("..")

if REPO_ROOT not in sys.path:
    sys.path.insert(0, REPO_ROOT)

import torch
from pathlib import Path
import pandas as pd
from tqdm.auto import tqdm

from ptx_decompiler.utils import get_device, supports_amp, is_colab

DEVICE = get_device()
print(f"Device: {DEVICE}")
print(f"AMP supported: {supports_amp(DEVICE)}")
if DEVICE.type == "mps":
    print("Apple Silicon detected — using MPS acceleration")
    print("Tip: MPS is ~3-5x faster than CPU for training")

In [ ]:
# ======================= Paths =======================
# Colab: data on Drive or in /content
# Local Mac: data in repo root (generated by notebook 01)

if IN_COLAB:
    # Option 1: Google Drive (persistent)
    # from google.colab import drive; drive.mount("/content/drive")
    # DATA_PATH = "/content/drive/MyDrive/NeuralPTX/dataset_100k.parquet"
    # SAVE_DIR  = Path("/content/drive/MyDrive/NeuralPTX/checkpoints")

    # Option 2: Local to Colab (faster I/O, lost on disconnect)
    DATA_PATH = os.path.join(REPO_ROOT, "dataset_100k.parquet")
    SAVE_DIR  = Path(REPO_ROOT) / "checkpoints"
else:
    # Local Mac / Linux
    DATA_PATH = os.path.join(REPO_ROOT, "dataset_100k.parquet")
    SAVE_DIR  = Path(REPO_ROOT) / "checkpoints"

print(f"Data: {DATA_PATH}")
print(f"Checkpoints: {SAVE_DIR}")
assert os.path.exists(DATA_PATH), f"Dataset not found at {DATA_PATH} — run notebook 01 first!"

## Load Data & Build Tokenizers

In [ ]:
from ptx_decompiler.data.dataset import load_parquet_for_training, collate_pad_batch, CurriculumSampler, PTXASTDataset
from ptx_decompiler.tokenizer import PTXTokenizer, ASTTokenizer

df = pd.read_parquet(DATA_PATH)
print(f"Loaded {len(df):,} samples")

ptx_tokenizer = PTXTokenizer(max_vocab_size=2000)
print("Building PTX vocabulary...")
ptx_tokenizer.build_vocab(df["ptx_normalized"].tolist())
print(f"PTX vocab size: {len(ptx_tokenizer)}")

ast_tokenizer = ASTTokenizer()
print(f"AST vocab size: {len(ast_tokenizer)}")

train_ds, val_ds = load_parquet_for_training(
    DATA_PATH, ptx_tokenizer, ast_tokenizer, train_ratio=0.9, seed=42
)
print(f"Train: {len(train_ds):,} | Val: {len(val_ds):,}")

curriculum_sampler = CurriculumSampler(train_ds, shuffle=True, seed=42)

# Batch size: smaller on MPS/CPU to avoid OOM
BATCH_SIZE = 128 if DEVICE.type == "cuda" else 64

train_loader = torch.utils.data.DataLoader(
    train_ds,
    batch_size=BATCH_SIZE,
    sampler=curriculum_sampler,
    collate_fn=lambda b: collate_pad_batch(b, ptx_tokenizer.pad_id, ast_tokenizer.pad_id),
    num_workers=0,
)
val_loader = torch.utils.data.DataLoader(
    val_ds,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=lambda b: collate_pad_batch(b, ptx_tokenizer.pad_id, ast_tokenizer.pad_id),
    num_workers=0,
)
print(f"Batch size: {BATCH_SIZE} | Train batches: {len(train_loader)} | Val batches: {len(val_loader)}")

## Build Model & Trainer

In [ ]:
from ptx_decompiler.model import PTXDecompilerModel
from ptx_decompiler.training import Trainer, get_cosine_schedule_with_warmup

# Map PTX tokens → AST vocab for copy mechanism
ptx_to_ast = torch.full((len(ptx_tokenizer),), -1, dtype=torch.long)
for tok, ptx_id in ptx_tokenizer.vocab.items():
    if tok in ast_tokenizer.vocab:
        ptx_to_ast[ptx_id] = ast_tokenizer.vocab[tok]

model = PTXDecompilerModel(
    ptx_vocab_size=len(ptx_tokenizer),
    ast_vocab_size=len(ast_tokenizer),
    d_model=256,
    n_heads=8,
    d_ff=1024,
    encoder_layers=6,
    decoder_layers=6,
    dropout=0.1,
    use_copy=True,
    ptx_to_ast_map=ptx_to_ast,
).to(DEVICE)

NUM_EPOCHS = 30
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=0.01)
num_steps = len(train_loader) * NUM_EPOCHS
scheduler = get_cosine_schedule_with_warmup(
    optimizer, num_warmup_steps=1000, num_training_steps=num_steps
)

trainer = Trainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    scheduler=scheduler,
    device=DEVICE,
    pad_id_ast=ast_tokenizer.pad_id,
    eos_id_ast=ast_tokenizer.eos_id,
    label_smoothing=0.1,
    use_amp=True,                  # auto-disabled on MPS/CPU by Trainer
    curriculum_sampler=curriculum_sampler,
    save_dir=SAVE_DIR,
    use_wandb=False,
)

print(f"Model parameters: {model.count_parameters():,}")
print(f"AMP enabled: {trainer.use_amp}")
print(f"Training on: {DEVICE}")

## Train

In [ ]:
trainer.train(num_epochs=NUM_EPOCHS)

In [ ]:
# Save final checkpoint
SAVE_DIR.mkdir(parents=True, exist_ok=True)
final_path = SAVE_DIR / "checkpoint_final.pt"
torch.save({"model": model.state_dict(), "epoch": NUM_EPOCHS - 1}, final_path)
print(f"Saved final checkpoint to {final_path}")